In [1]:
import re
import pandas as pd
import requests 
import json

### Starting of classification Tag evaluation API

In [16]:
merged_corpus = pd.read_csv('/home/ifte/resource/docomo_dic/data/merged_corpus.csv')

In [18]:
merged_corpus.iloc[519].title

'BMWのレンタカー利用時に事故や車両の故障・不具合が発生した場合はどこに連絡すればいいですか？ | よくあるご質問| カーシェアならdカーシェア | ドコモのカーシェアリングサービス'

In [2]:
raw_corpus = pd.read_csv('/home/ifte/resource/docomo_dic/data/corpus.csv')
# raw_corpus = pd.read_csv('/home/iftekhar/docomodezwhu/5495/data/corpus.csv')

### Converter DocID => URLs

In [18]:
# defining the api-endpoint  
API_ENDPOINT = "http://127.0.0.1:5000/api/v1.0/predict/"
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
result = []
data = {
# "output_dir": "/home/ifte/resource/docomo_dic/model", 
"output_dir": "/home/iftekhar/docomodezwhu/model", 
"agent_id": -1, 
"query": "メールサービスの利用にあたっての注意点を教えてください。",
"classification": "dトラベル",
"max_answer": 5, 
# "data_dir": "/home/ifte/resource/docomo_dic/data", 
"data_dir": "/home/iftekhar/docomodezwhu/data", 
"keyphrase_extraction": 0
}

r = requests.post(url = API_ENDPOINT, data=json.dumps(data), headers = headers) 



In [19]:
# json.loads(r.text)
for i in json.loads(r.text)['page_relevance']:
    print(list(set(raw_corpus[raw_corpus['page'] == i['doc_id']].url.values))[0], i['doc_id'])


https://travel.dmkt-sp.jp/faq/176/ 337
https://travel.dmkt-sp.jp/faq/106/ 323
https://travel.dmkt-sp.jp/faq/21/ 129
https://travel.dmkt-sp.jp/faq/82 396
https://travel.dmkt-sp.jp/campaign/point_car 221


In [17]:
# json.loads(r.text)
for i in json.loads(r.text)['page_relevance']:
    print(list(set(raw_corpus[raw_corpus['page'] == i['doc_id']].url.values))[0], i['doc_id'])

    

https://travel.dmkt-sp.jp/faq/176/ 337
https://travel.dmkt-sp.jp/faq/106/ 323
https://travel.dmkt-sp.jp/faq/21/ 129
https://travel.dmkt-sp.jp/faq/82 396
https://travel.dmkt-sp.jp/campaign/point_car 221


In [40]:
list(set(raw_corpus[raw_corpus['page'] == 760].url.values))[0]


'https://travel.dmkt-sp.jp/faq/3/'

In [52]:
' '.join(raw_corpus[raw_corpus['page'] == 760].text.values)

'宿泊代金の支払方法について教えてください。 宿泊代金の支払方法について教えてください。|よくあるご質問 Q.宿泊代金の支払方法について教えてください。 A. 商品によって異なります。以下をご確認ください。 【宿提供マークのついているプラン】 宿泊当日に宿泊施設現地でお支払いください。 現金以外のお支払い方法の可否については宿泊施設へお問い合わせください。 ※各施設で利用できるクレジットカードについては各施設の「宿・ホテル詳細」をご覧下さい。 【手配マークのついているプラン】／【エースJTBマークのついているプラン】／【サン＆サンマークのついているプラン】 ドコモ払い、dカード払い、クレジットカード払いをご利用いただけます。 ※ｄトラベルでは海外発行のクレジットカードのご利用はできません。あらかじめご了承ください。 ※iD、d払いはご利用いただけません。 キーワード検索 お困りの内容 (c)NTT DOCOMO, INC. All Rights Reserved.'

In [44]:
with open('/home/iftekhar/docomodezwhu/data/processed_texts.txt') as f:
    data = f.readlines()

In [51]:
data[759]

'ある ご 質問 検索 結果 ある ご 質問 ドコモ dトラベル 宿泊代金 支払 方法 教える 宿泊代金 支払 方法 教える ある ご 質問 q 宿泊 料金 支払 方法 教える 商品 異なる ご 確認 くださる 旅館 提供 マーク つく プラン 宿泊 当日 宿泊施設 現地 お 支払い くださる 金 お 支払い 方法 可否 宿泊施設 お 問い合わせ 各 施設 利用 できる クレジットカード 各 施設 旅館 ホテル 詳細 ご覧 下さる 手配 マーク つく プラン エース jtb マーク つく プラン サン サンマーク つく プラン ドコモ 払う d カード払い クレジットカード 払い ご 利用 いただける dトラベル 外国 発行 クレジットカード ご 利用 できる ご 了承 id d払い ご 利用 いただける\n'

In [3]:
def mean_reciprocal_rank_score(actual_value, predicted_values):
    if not actual_value:
        return 0
    pos = 0
    val = 0
    for i in predicted_values:
        if i == actual_value and pos == 0:
            val = 1
            break
        elif i == actual_value and pos == 1:
            val = 0.5
            break
        elif i == actual_value and pos == 2:
            val = 0.33
            break
        else:
            val = 0
        pos += 1

    return val

def cut_string(text):
    try:
        # print(text[:text.index("?")])
        return text[:text.index("?")]
    except ValueError:
        # print(text)
        return text

In [4]:
# test_data = '/home/iftekhar/Documents/test_data_docomo_07.12.2020_red.csv'
# test_data = '/home/iftekhar/Documents/test_data_docomo_07.12.2020.csv'
test_data = '/home/ifte/Documents/test_data_docomo_nai_form.csv'
# test_data = '/home/ifte/Documents/test_data_few_exceptional.csv'
# test_data = '/home/iftekhar/Documents/test_data_docomo_21.12.2020.csv'
test_df = pd.read_csv(test_data)

In [5]:
test_df.rename(columns={'correct_url': 'url'}, inplace=True)

In [6]:
test_df.url = test_df.url.apply(lambda x: cut_string(x))

In [ ]:
test_df.url

In [ ]:
# defining the api-endpoint  
API_ENDPOINT = "http://127.0.0.1:5000/api/v1.0/predict/"
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
result = []
i = 1
for index, col in test_df.iterrows():
    # data to be sent to api 
    data = {'output_dir':'/home/ifte/resource/docomo_dic/model', 
            'data_dir':'/home/ifte/resource/docomo_dic/data', 
            'agent_id':-1, 
            'query':col['question'],
            'classification': col['tag'],
            'max_answer':5
           } 

#     data = {'output_dir':'/home/iftekhar/docomodezwhu/5495/model', 
#             'data_dir':'/home/iftekhar/docomodezwhu/5495/data', 
#             'agent_id':-1, 
#             'query':col['question'],
#             'classification': col['tag'],
#             'max_answer':5
#            } 
    
    r = requests.post(url = API_ENDPOINT, data=json.dumps(data), headers = headers)
    print('Request count:', i)
    i += 1
    result.append(r.text)
    

In [ ]:
result

In [16]:
from collections import defaultdict
saver = defaultdict(list)

j = 0
for items in result:
    for i in json.loads(items)['page_relevance']:
        saver[j].append(list(set(raw_corpus[raw_corpus['page'] == i['doc_id']].url.values))[0])
        saver[str(str(j)+'doc_id')].append(i['doc_id'])
    j += 1
        

In [ ]:
saver

In [17]:
collector = defaultdict(list)
i = 0

for index, col in test_df.iterrows():
    collector['question'].append(col['question'])
    collector['tag'].append(col['tag'])
    collector['correct_url'].append(col['url'])
    collector_correct = []
    try:
        collector_correct = list(set(raw_corpus[raw_corpus['url'] == col['url']].page.values))[0]
    except IndexError:
        continue
        pass
    collector['correct_id'].append(collector_correct)
    collector['returned_id'].append(saver[str(str(i)+'doc_id')])
    score = mean_reciprocal_rank_score(collector_correct, saver[str(str(i)+'doc_id')])
    if score:
        collector['mrr_score'].append(score)
    else:
        try:
            url_loc = saver[i].index(col['url'])
            if url_loc == 1:
                collector['mrr_score'].append(0.5)
            elif url_loc == 2:
                collector['mrr_score'].append(0.33)
            elif url_loc == 0:
                collector['mrr_score'].append(1)
            elif url_loc >= 3:
                collector['mrr_score'].append('LOW_RANK')
    
        except ValueError:
            collector['mrr_score'].append('NOT_MATCHED')
            pass

    collector['urls'].append(saver[i])
#         print('ok ', i)
    i += 1

    

In [10]:
collector.keys()

dict_keys(['question', 'tag', 'correct_url', 'correct_id', 'returned_id', 'mrr_score', 'urls'])

In [18]:
new_df = pd.DataFrame.from_dict(collector, orient='index').T

In [12]:
new_df.head(10)

,question,tag,correct_url,correct_id,returned_id,mrr_score,urls
0,予約しましたが、メールが届きません。,dトラベル,https://travel.dmkt-sp.jp/faq/58/,951,"[951, 944, 943, 820, 1172]",1,"[https://travel.dmkt-sp.jp/faq/58/, https://tr..."
1,予約の取消（キャンセル）が出来ません。,dトラベル,https://travel.dmkt-sp.jp/faq/64/,1013,"[1013, 1158, 525, 335, 372]",1,"[https://travel.dmkt-sp.jp/faq/64/, nan, https..."
2,dマーケットを利用しているが、dマーケットマスターチャレンジの対象となっていません。,dトラベル,https://travel.dmkt-sp.jp/faq/98/,852,"[852, 939, 792, 42, 765]",1,"[https://travel.dmkt-sp.jp/faq/98/, https://tr..."
3,メールサービスが届きません。,dトラベル,https://travel.dmkt-sp.jp/faq/104/,944,"[944, 943, 820, 62, 22]",1,"[https://travel.dmkt-sp.jp/faq/104/, https://t..."
4,購入しましたが、購入完了メールが届きません。,dトラベル,https://travel.dmkt-sp.jp/faq/121/,820,"[820, 978, 976, 1172, 679]",1,"[https://travel.dmkt-sp.jp/faq/121/, https://t..."
5,スマートフォン紛失のため、スマートフォンを当日持って行けません。,dトラベル,https://travel.dmkt-sp.jp/faq/130/,1039,"[1039, 989, 1041, 292, 243]",1,"[https://travel.dmkt-sp.jp/faq/130/, https://t..."
6,国内航空券の代金に対し、ポイントが足りません。,dトラベル,https://travel.dmkt-sp.jp/faq/159/,964,"[964, 958, 816, 1272, 1278]",1,"[https://travel.dmkt-sp.jp/faq/159/, https://t..."
7,国内航空券のポイントが進呈されません。,dトラベル,https://travel.dmkt-sp.jp/faq/158/,961,"[958, 961, 783, 644, 619]",0.5,"[https://travel.dmkt-sp.jp/faq/157/, https://t..."
8,ポイントが利用できません。,dトラベル,https://travel.dmkt-sp.jp/faq/14/,769,"[924, 991, 816, 992, 769]",LOW_RANK,"[https://travel.dmkt-sp.jp/faq/27/, https://tr..."
9,くじで獲得したポイントが反映されません。,dマーケット,https://d.dmkt-sp.jp/common/qa/category/kuji/i...,135,"[135, 66, 121, 94, 87]",1,[https://d.dmkt-sp.jp/common/qa/category/kuji/...


In [19]:
new_df.columns

Index(['question', 'tag', 'correct_url', 'correct_id', 'returned_id',
       'mrr_score', 'urls'],
      dtype='object')

In [20]:
new_df.to_csv('nai_form_with_classification_tag_23.12.2020.csv')

## corpus analysis

In [21]:
# 会員登録時にsmsにて電話番号認証をするのはなぜですか
merged = pd.read_csv('/home/ifte/resource/docomo_dic/data/corpus.csv')

In [ ]:
merged[merged.page == 591].text.values

In [10]:
merged_all = pd.read_csv('/home/ifte/resource/docomo_dic/data/merged_corpus.csv')

In [ ]:
merged_all[merged_all.page == 591].text.values

In [13]:
merged_all[merged_all.page == 591].index

Int64Index([591], dtype='int64')

In [14]:
processed = '/home/ifte/resource/docomo_dic/data/processed_texts.txt'
with open(processed) as f:
    data = f.readlines()

In [20]:
for index, line in enumerate(data):
    if index == 146:
        print(line)

特典 ご 注意事項 dマーケット 特典 ご 注意事項 マスチャレ 特典 ポイント ご 注意事項 キャンペーン 内容 マスターチャレンジパスポート 獲得 dマーケット マスター お客様 本 キャンペーン 対象 サービス お 買い物  dポイント  期間 用途 限定 進呈 dマーケット マスター チャレンジャー なす 通常 d ポイント キャンペーン 対象 なる お 買い物 期間 毎月 毎月 末 キャンペーン 対象 サービス  dショッピング   dブック   dトラベル  d fashion  dミュージック   dデリバリー   dカーシェア  dtv レンタル  dアニメストア  レンタル グッズ  dエンジョイパス   dフォト   dゲーム  d ゲームプレイ チケット  dミュージック  dtv dアニメストア 月額 サービス 対象外  dポイント  期間 用途 限定 進呈 対象者 dマーケット マスター お客様 キャンペーン期間 対象 サービス 対象商品 コンテンツ ご 購入 マスターチャレンジパスポート 獲得 商品 コンテンツ ご 購入 ドコモ ご 契約 携帯電話 回線 d アカウント  dポイント  期間 用途 限定 進呈 マスターチャレンジパスポート dマーケット マスター お客様 マスターチャレンジパスポート 獲得 ダクト 本 キャンペーン 対象 サービス お 買い物 最大 倍  dポイント  期間 用途 限定 進呈 パスポート 獲得 同 月内 購入 商品 point-up 対象 なる パスポート 獲得 いただく 翌月 獲得 不要 獲得 方法 詳細 ご 確認 くださる 同 月内 商品 購入 マスターチャレンジパスポート 獲得 ポイント 進呈 対象 なる dマーケット マスター お客様 パスポート 獲得 いただける  dポイント  期間 用途 限定 進呈 進呈 期限 進呈 ご 購入 商品 コンテンツ お 支払い 確定 翌月 中旬 頃 ご 利用 期限  dポイント  期間 用途 限定 進呈 進呈 末 商品 注文 商品 発送 お 支払い 確定 当該 商品 かかる ポイント 中旬 頃 進呈 雅語 利用 期限 失効 なる 本 キャンペーン 進呈 ポイント 上限 ポイント 期間 用途 限定 なる 進呈 ポイント 数 計算 原則 お 支払い 確定 翌月 初旬 頃